In [2]:
## CODE ADAPTED FROM: https://medium.com/analytics-vidhya/topic-modeling-using-gensim-lda-in-python-48eaa2344920

import numpy as np
import pandas as pd
import re
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

truncated = pd.read_csv("FINAL_DATASET_TRUNC.csv", index_col=0)
truncated.head()

# FOR EACH ORGANIZATION, MERGE THE TEXTS CORRESPONDING TO THAT ORGANIZATION

unique_orgs = list(truncated.organization.unique())
unique_orgs

text_collection = []

for org in unique_orgs:
    merged_text = ' '.join(str(x) for x in truncated.Text_translated[truncated.organization == org])
    text_collection.append(merged_text)
    

print(len(unique_orgs), len(text_collection))

df = pd.DataFrame(list(zip(list(unique_orgs), text_collection)), columns=['organization', 'Text'])

df

df2 = df.rename({'Text': 'merged_texts'}, axis=1)
df2

### Americanizing all the words (i.e. transforming all words that are originally in British English to American English)

import requests

def americanize(string):
    url ="https://raw.githubusercontent.com/hyperreality/American-British-English-Translator/master/data/british_spellings.json"
    british_to_american_dict = dict(requests.get(url).json())    

    for british_spelling, american_spelling in british_to_american_dict.items():
        string = string.replace(british_spelling, american_spelling)
  
    return string

# Apply the americanize function to the dataframe
df['Text'] = df['Text'].apply(func = americanize)


## Stopwords Removal section

# bring new stopwords from the junk topics of the previous iterations 
# (16 topic model, 5 junk topics, top 1% words from each of the junk topics ~ 1220 words * 5 = 6100 words)

junk = pd.read_excel("junk_words_model_16t_top_1220.xlsx", index_col=0)
initial_junk_words = list(junk.word)
print(len(initial_junk_words))
initial_junk_words_SET = list(set(initial_junk_words))
print(len(initial_junk_words_SET))



# REMOVE STOP WORDS
# NLTK Stop words
from nltk.corpus import stopwords
stop_words_eng = list(stopwords.words('english'))
stop_words_fr = list(stopwords.words('french'))
stop_words_sp = list(stopwords.words('spanish'))
stop_words_de = list(stopwords.words('german'))
stop_words = stop_words_eng + stop_words_fr + stop_words_sp + stop_words_de
stop_words.extend(["retrieved", "retrieve", "january","february","march","april", "may","june", "july", 
                   "august","september", "october", "november", "december","wikipedia", "facebook", 
                   "instagram", "linkedIn","used", "u", "U", "b", "lat", "lon", 
                   "archived", "archive", "download","about", "events", "careers", "contact"])

# add the junk words
stop_words.extend(initial_junk_words_SET)

# Convert to list 
data = df.Text.values.tolist()  
# Remove new line characters 
data = [re.sub(r'\s+', ' ', sent) for sent in data]  
# Remove distracting single quotes 
data = [re.sub(r"\'", "", sent) for sent in data]  
pprint(data[:1])

# TRANSFORM TEXT TO LIST OF INDIVIDUAL WORDS

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) #deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

## Extra Step:
### Removing words with less than 2 characters

def remove_two_letter_words(words):
    result = []
    for word in words:
        if len(word) > 2:
            result.append(word)
    return result

for i in range(len(data_words)):
    data_words[i] = remove_two_letter_words(data_words[i])
    

# sanity check:
local_min = []
for i in range(len(data_words)):
    word_list = data_words[i]
    local_min.append(min([len(word) for word in word_list]))
print(min(local_min))

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

# ?gensim.models.Phrases

# ?gensim.models.phrases.original_scorer

# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner']) # SpaCy documentation: https://spacy.io/usage/spacy-101

def lemmatization(texts):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc])
    return texts_out

# CALL THE FUNCTIONS IN ORDER:

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)

print(data_lemmatized[:1])

# started 15:42 -> ended 15:45

# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  # doc2bow = document to bag-of-words
# View 
print(corpus[:1])

# Gensim creates unique id for each word in the document. 
# The mapping of the corpus is as follows: every pair (a, b) in fact represents (word_id, word_frequency). 

# If you want to see what word corresponds to a given id, 
# then pass the id as a key to dictionary. 
# Example: 
# id2word[4]

# to see the word frequency distribution in a given document, run this:
# [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

1118 1118
6100
1229
['2 Degrees Investing Initiative - World Benchmarking Alliance Skip to content '
 'Building a movement How it started Mission and vision Alliance About the '
 'Alliance Join the Alliance Meet our Allies How we work Seven systems '
 'transformations Theory of change Benchmarking SDG2000 Benchmarks Social '
 'Gender Benchmark Corporate Human Rights Benchmark Food and Agriculture '
 'Seafood Stewardship Index Access to Seeds Index Climate and Energy Just '
 'transition Nature Digital Inclusion Financial System Companies About us Team '
 'Careers and culture Funding partners Governance Ambassadors Finances Contact '
 'Content News and events Keeping you updated on our work Research and '
 'publications Sharing our latest data, findings and insights Impact Building '
 'a movement In the media Allies Companies This site makes use of third-party '
 'cookies for analytics. By continuing to use the site, you accept the use of '
 'cookies. Our privacy statement Accept France 

[['degrees', 'investing', 'initiative', 'world', 'benchmarking', 'alliance', 'skip', 'to', 'content', 'building', 'movement', 'how', 'it', 'started', 'mission', 'and', 'vision', 'alliance', 'about', 'the', 'alliance', 'join', 'the', 'alliance', 'meet', 'our', 'allies', 'how', 'we', 'work', 'seven', 'systems', 'transformations', 'theory', 'of', 'change', 'benchmarking', 'sdg', 'benchmarks', 'social', 'gender', 'benchmark', 'corporate', 'human', 'rights', 'benchmark', 'food', 'and', 'agriculture', 'seafood', 'stewardship', 'index', 'access', 'to', 'seeds', 'index', 'climate', 'and', 'energy', 'just', 'transition', 'nature', 'digital', 'inclusion', 'financial', 'system', 'companies', 'about', 'us', 'team', 'careers', 'and', 'culture', 'funding', 'partners', 'governance', 'ambassadors', 'finances', 'contact', 'content', 'news', 'and', 'events', 'keeping', 'you', 'updated', 'on', 'our', 'work', 'research', 'and', 'publications', 'sharing', 'our', 'latest', 'data', 'findings', 'and', 'insigh

3
['degrees', 'investing', 'initiative', 'world', 'benchmarking', 'alliance', 'skip_content', 'building', 'movement', 'how', 'started', 'mission', 'and', 'vision', 'alliance', 'about', 'the', 'alliance', 'join', 'the', 'alliance', 'meet', 'our', 'allies', 'how', 'work', 'seven', 'systems', 'transformations', 'theory', 'change', 'benchmarking', 'sdg', 'benchmarks', 'social', 'gender', 'benchmark', 'corporate', 'human_rights', 'benchmark', 'food', 'and', 'agriculture', 'seafood', 'stewardship', 'index', 'access', 'seeds', 'index', 'climate', 'and', 'energy', 'just_transition', 'nature', 'digital', 'inclusion', 'financial', 'system', 'companies', 'about', 'team', 'careers', 'and', 'culture', 'funding', 'partners', 'governance', 'ambassadors', 'finances', 'contact', 'content', 'news', 'and', 'events', 'keeping', 'you', 'updated', 'our', 'work', 'research', 'and', 'publications', 'sharing', 'our', 'latest', 'data', 'findings', 'and', 'insights', 'impact', 'building', 'movement', 'the', 'med

[['degree', 'invest', 'initiative', 'world', 'benchmarking', 'alliance', 'skip_content', 'building', 'movement', 'start', 'mission', 'vision', 'alliance', 'alliance', 'join', 'alliance', 'meet', 'ally', 'work', 'seven', 'system', 'transformation', 'theory', 'change', 'benchmarke', 'sdg', 'benchmark', 'social', 'gender', 'benchmark', 'corporate', 'human_rights', 'benchmark', 'food', 'agriculture', 'seafood', 'stewardship', 'index', 'access', 'seed', 'index', 'climate', 'energy', 'transition', 'nature', 'digital', 'inclusion', 'financial', 'system', 'company', 'team', 'culture', 'funding', 'partner', 'governance', 'ambassador', 'finance', 'content', 'news', 'keep', 'update', 'work', 'research', 'publication', 'share', 'late', 'datum', 'finding', 'insight', 'impact', 'building', 'movement', 'medium', 'ally', 'company', 'site', 'make', 'use', 'third_party', 'cookie', 'analytic', 'continue', 'use', 'site', 'accept', 'use', 'cookie', 'privacy', 'statement', 'accept', 'france', 'degree', 'inv

[[(0, 2), (1, 3), (2, 3), (3, 2), (4, 2), (5, 1), (6, 1), (7, 2), (8, 2), (9, 1), (10, 6), (11, 3), (12, 1), (13, 2), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 14), (20, 3), (21, 7), (22, 7), (23, 2), (24, 2), (25, 8), (26, 2), (27, 4), (28, 4), (29, 7), (30, 2), (31, 1), (32, 3), (33, 1), (34, 2), (35, 2), (36, 4), (37, 2), (38, 2), (39, 2), (40, 1), (41, 1), (42, 1), (43, 2), (44, 4), (45, 2), (46, 2), (47, 1), (48, 4), (49, 1), (50, 1), (51, 2), (52, 8), (53, 2), (54, 2), (55, 9), (56, 2), (57, 1), (58, 2), (59, 3), (60, 1), (61, 5), (62, 14), (63, 1), (64, 2), (65, 2), (66, 9), (67, 2), (68, 2), (69, 2), (70, 2), (71, 1), (72, 3), (73, 2), (74, 1), (75, 7), (76, 1), (77, 2), (78, 1), (79, 1), (80, 4), (81, 1), (82, 2), (83, 1), (84, 1), (85, 4), (86, 2), (87, 1), (88, 5), (89, 2), (90, 1), (91, 7), (92, 1), (93, 10), (94, 5), (95, 1), (96, 2), (97, 1), (98, 5), (99, 2), (100, 2), (101, 4), (102, 1), (103, 1), (104, 5), (105, 4), (106, 2), (107, 3), (108, 1), (109, 2), (110,

In [3]:
print(id2word[4])

adaptation


In [5]:
corpora.MmCorpus.serialize("./Corpora/V2_LDA_corpus_object", corpus)

In [6]:
loaded_corp = corpora.MmCorpus("./Corpora/V2_LDA_corpus_object")
loaded_corp

In [8]:
print(loaded_corp[:1])

In [12]:
mm = corpora.MmCorpus("./Corpora/V2_LDA_corpus_object")
print(mm[0])

[(0, 2.0), (1, 3.0), (2, 3.0), (3, 2.0), (4, 2.0), (5, 1.0), (6, 1.0), (7, 2.0), (8, 2.0), (9, 1.0), (10, 6.0), (11, 3.0), (12, 1.0), (13, 2.0), (14, 1.0), (15, 2.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 14.0), (20, 3.0), (21, 7.0), (22, 7.0), (23, 2.0), (24, 2.0), (25, 8.0), (26, 2.0), (27, 4.0), (28, 4.0), (29, 7.0), (30, 2.0), (31, 1.0), (32, 3.0), (33, 1.0), (34, 2.0), (35, 2.0), (36, 4.0), (37, 2.0), (38, 2.0), (39, 2.0), (40, 1.0), (41, 1.0), (42, 1.0), (43, 2.0), (44, 4.0), (45, 2.0), (46, 2.0), (47, 1.0), (48, 4.0), (49, 1.0), (50, 1.0), (51, 2.0), (52, 8.0), (53, 2.0), (54, 2.0), (55, 9.0), (56, 2.0), (57, 1.0), (58, 2.0), (59, 3.0), (60, 1.0), (61, 5.0), (62, 14.0), (63, 1.0), (64, 2.0), (65, 2.0), (66, 9.0), (67, 2.0), (68, 2.0), (69, 2.0), (70, 2.0), (71, 1.0), (72, 3.0), (73, 2.0), (74, 1.0), (75, 7.0), (76, 1.0), (77, 2.0), (78, 1.0), (79, 1.0), (80, 4.0), (81, 1.0), (82, 2.0), (83, 1.0), (84, 1.0), (85, 4.0), (86, 2.0), (87, 1.0), (88, 5.0), (89, 2.0), (90, 1.0), (91, 7

In [13]:
id2word.save("./Corpora/V2_LDA_id2word_object")

In [ ]:
# We will now work with df2, a dataframe that contains the organization names + all texts for a specific organization, merged


In [14]:
from gensim import corpora, models, similarities

lda_model_18t = models.LdaModel.load("./V2 Final Models/18 topics/V2_18topics_model")

In [15]:
document_topics_18 = list(lda_model_18t.get_document_topics(corpus, minimum_probability=0))

In [16]:
document_topics_18

[[(0, 1.711062e-05),
  (1, 4.2032957e-06),
  (2, 4.2896713e-06),
  (3, 0.01321456),
  (4, 0.009269178),
  (5, 6.1873807e-06),
  (6, 8.873187e-05),
  (7, 0.14144377),
  (8, 0.015887493),
  (9, 0.00017756561),
  (10, 9.43792e-05),
  (11, 1.7587976e-05),
  (12, 0.09474926),
  (13, 0.24832956),
  (14, 0.25009763),
  (15, 0.004390191),
  (16, 4.1692897e-06),
  (17, 0.22220416)],
 [(0, 1.241985e-05),
  (1, 3.0509882e-06),
  (2, 3.1136847e-06),
  (3, 0.074350305),
  (4, 0.003756821),
  (5, 4.4911485e-06),
  (6, 0.18900569),
  (7, 0.00014401221),
  (8, 0.0071275625),
  (9, 0.015466278),
  (10, 6.848496e-05),
  (11, 1.2766341e-05),
  (12, 0.052269913),
  (13, 0.39784393),
  (14, 0.00026345206),
  (15, 0.24625401),
  (16, 3.0263047e-06),
  (17, 0.013410677)],
 [(0, 8.190532e-06),
  (1, 2.0120387e-06),
  (2, 2.053385e-06),
  (3, 0.00014405731),
  (4, 1.565701e-05),
  (5, 2.961783e-06),
  (6, 0.060152993),
  (7, 9.433302e-05),
  (8, 0.047446296),
  (9, 0.21638647),
  (10, 4.5170862e-05),
  (11, 8.

In [17]:
def transform_document_topic_list(list_doc_topics, index_column):
    result = []
    most_prevalent_topic = []
    for i in range(len(list_doc_topics)):
        probabilities = []
        maximum = -1
        index = -1
        topic_distrib = list_doc_topics[i]
        L = len(topic_distrib)
        for i in range(len(topic_distrib)):
            t = topic_distrib[i]
            probabilities.append(t[1])
            if t[1] > maximum:
                maximum = t[1]
                index = i + 1
        probabilities = tuple(probabilities)
        result.append(probabilities)
        most_prevalent_topic.append(index)
        
    df = pd.DataFrame(result, columns = [f"Topic{i}" for i in range(1, L+1)])
    df['most_prevalent_topic'] = most_prevalent_topic
    df['organization'] = index_column
    return df

In [18]:
E1 = transform_document_topic_list(document_topics_18, list(df2.organization))
E1

cols = E1.columns.tolist()
cols = cols[-1:] + cols[:-1]
E1 = E1[cols]
E1

,organization,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,most_prevalent_topic
0,2° investing initiative (2°ii),0.000017,0.000004,0.000004,0.013215,0.009269,0.000006,0.000089,0.141444,0.015887,0.000178,0.000094,0.000018,0.094749,0.248330,0.250098,0.004390,0.000004,0.222204,15
1,A SEED Europe,0.000012,0.000003,0.000003,0.074350,0.003757,0.000004,0.189006,0.000144,0.007128,0.015466,0.000068,0.000013,0.052270,0.397844,0.000263,0.246254,0.000003,0.013411,14
2,A Sud Ecology and Cooperation - Onlus,0.000008,0.000002,0.000002,0.000144,0.000016,0.000003,0.060153,0.000094,0.047446,0.216386,0.000045,0.000008,0.021028,0.286143,0.111477,0.256998,0.000002,0.000044,14
3,ACT Alliance - Action by Churches Together,0.000006,0.000002,0.000002,0.008190,0.000012,0.000002,0.000033,0.000073,0.023083,0.199586,0.029967,0.004066,0.522229,0.084637,0.121968,0.006107,0.000002,0.000034,13
4,Abibimman Foundation,0.000010,0.000002,0.000002,0.027990,0.000018,0.000003,0.047113,0.034649,0.002725,0.040752,0.005691,0.000010,0.000216,0.621083,0.172883,0.046797,0.000002,0.000051,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,ZOI Environment Network,0.000019,0.000005,0.000005,0.000319,0.000036,0.000007,0.003195,0.000214,0.000390,0.007865,0.000103,0.000019,0.000401,0.282942,0.493778,0.210600,0.000005,0.000100,15
1114,Zhenjiang Green Sanshan Environmental Public W...,0.000011,0.000003,0.000003,0.009154,0.000021,0.000004,0.000057,0.012795,0.213201,0.406350,0.000060,0.000011,0.010802,0.346956,0.000233,0.000277,0.000003,0.000058,10
1115,atmosfair gGmbH,0.000006,0.000002,0.000002,0.007350,0.000012,0.000002,0.007171,0.661009,0.009456,0.000064,0.000034,0.000006,0.032657,0.104741,0.170982,0.006472,0.000002,0.000033,8
1116,iuventum e.V.,0.000007,0.000002,0.000002,0.002980,0.000013,0.000002,0.000035,0.012312,0.024148,0.286403,0.096842,0.000007,0.425230,0.098294,0.002206,0.040134,0.000002,0.011384,13


In [20]:
NGOs = pd.read_csv("NGOs_COP2019_complete.csv")
NGOs = NGOs[["name", "entity_type", "coordinator", "city", "country", "constituency"]]

IGOs = pd.read_csv("IGOs_COP2019_complete.csv")
IGOs = IGOs[["name", "entity_type", "coordinator", "city", "country"]]

In [21]:
merge_all = pd.concat([NGOs, IGOs], ignore_index=True)
merge_all

,name,entity_type,coordinator,city,country,constituency
0,Technical Centre for Agricultural and Rural Co...,Non-governmental organizations,Mr. Michael Hailu,Wageningen,Netherlands,NaN
1,The European Organisation for the Exploitation...,Non-governmental organizations,NaN,Darmstadt,Germany,NaN
2,2° investing initiative (2°ii),Non-governmental organizations,Mr. Thibaut Ghirardi,Paris,France,Constituency - Research and independent
3,A SEED Europe,Non-governmental organizations,Mr. Flip Vonk,Amsterdam,Netherlands,Constituency - Environmental
4,A Sud Ecology and Cooperation - Onlus,Non-governmental organizations,Ms. Lucie Greyl,Rome,Italy,Constituency - Environmental
...,...,...,...,...,...,...
1113,Regional Community Forestry Training Center fo...,Intergovernmental organizations,Ms. Jennifer Jadin,Bangkok,Thailand,NaN
1114,Secretaria General Iberoamericana,Intergovernmental organizations,Ms. Maria Salvadora Ortiz,Madrid,Spain,NaN
1115,Secrétariat général de l’union du Maghreb Arabe,Intergovernmental organizations,Mme Faouzia Chakiri,Rabat,Morocco,NaN
1116,Secretariat of the Pacific Community,Intergovernmental organizations,Ms. Sylvie Michelle Goyet,Noumea,France,NaN


In [22]:
merge_all = merge_all.rename({'name': 'organization'}, axis = 1)
merge_all

,organization,entity_type,coordinator,city,country,constituency
0,Technical Centre for Agricultural and Rural Co...,Non-governmental organizations,Mr. Michael Hailu,Wageningen,Netherlands,NaN
1,The European Organisation for the Exploitation...,Non-governmental organizations,NaN,Darmstadt,Germany,NaN
2,2° investing initiative (2°ii),Non-governmental organizations,Mr. Thibaut Ghirardi,Paris,France,Constituency - Research and independent
3,A SEED Europe,Non-governmental organizations,Mr. Flip Vonk,Amsterdam,Netherlands,Constituency - Environmental
4,A Sud Ecology and Cooperation - Onlus,Non-governmental organizations,Ms. Lucie Greyl,Rome,Italy,Constituency - Environmental
...,...,...,...,...,...,...
1113,Regional Community Forestry Training Center fo...,Intergovernmental organizations,Ms. Jennifer Jadin,Bangkok,Thailand,NaN
1114,Secretaria General Iberoamericana,Intergovernmental organizations,Ms. Maria Salvadora Ortiz,Madrid,Spain,NaN
1115,Secrétariat général de l’union du Maghreb Arabe,Intergovernmental organizations,Mme Faouzia Chakiri,Rabat,Morocco,NaN
1116,Secretariat of the Pacific Community,Intergovernmental organizations,Ms. Sylvie Michelle Goyet,Noumea,France,NaN


In [24]:
D1 = pd.merge(E1, merge_all, on = "organization", how = "left")
D1

,organization,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic15,Topic16,Topic17,Topic18,most_prevalent_topic,entity_type,coordinator,city,country,constituency
0,2° investing initiative (2°ii),0.000017,0.000004,0.000004,0.013215,0.009269,0.000006,0.000089,0.141444,0.015887,...,0.250098,0.004390,0.000004,0.222204,15,Non-governmental organizations,Mr. Thibaut Ghirardi,Paris,France,Constituency - Research and independent
1,A SEED Europe,0.000012,0.000003,0.000003,0.074350,0.003757,0.000004,0.189006,0.000144,0.007128,...,0.000263,0.246254,0.000003,0.013411,14,Non-governmental organizations,Mr. Flip Vonk,Amsterdam,Netherlands,Constituency - Environmental
2,A Sud Ecology and Cooperation - Onlus,0.000008,0.000002,0.000002,0.000144,0.000016,0.000003,0.060153,0.000094,0.047446,...,0.111477,0.256998,0.000002,0.000044,14,Non-governmental organizations,Ms. Lucie Greyl,Rome,Italy,Constituency - Environmental
3,ACT Alliance - Action by Churches Together,0.000006,0.000002,0.000002,0.008190,0.000012,0.000002,0.000033,0.000073,0.023083,...,0.121968,0.006107,0.000002,0.000034,13,Non-governmental organizations,Mr. Thorsten Göbel,Geneva,Switzerland,Constituency - Environmental CAN
4,Abibimman Foundation,0.000010,0.000002,0.000002,0.027990,0.000018,0.000003,0.047113,0.034649,0.002725,...,0.172883,0.046797,0.000002,0.000051,14,Non-governmental organizations,Mr. Kenneth Nana Amoateng,Tena,Ghana,Constituency - Environmental CAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,ZOI Environment Network,0.000019,0.000005,0.000005,0.000319,0.000036,0.000007,0.003195,0.000214,0.000390,...,0.493778,0.210600,0.000005,0.000100,15,Non-governmental organizations,Mr. Viktor Novikov,Geneva,Switzerland,Constituency - Research and independent
1114,Zhenjiang Green Sanshan Environmental Public W...,0.000011,0.000003,0.000003,0.009154,0.000021,0.000004,0.000057,0.012795,0.213201,...,0.000233,0.000277,0.000003,0.000058,10,Non-governmental organizations,NaN,中国江苏省镇江市,China,Constituency - Environmental
1115,atmosfair gGmbH,0.000006,0.000002,0.000002,0.007350,0.000012,0.000002,0.007171,0.661009,0.009456,...,0.170982,0.006472,0.000002,0.000033,8,Non-governmental organizations,Ms. Cathleen Herrich,Berlin,Germany,Constituency - Environmental
1116,iuventum e.V.,0.000007,0.000002,0.000002,0.002980,0.000013,0.000002,0.000035,0.012312,0.024148,...,0.002206,0.040134,0.000002,0.011384,13,Non-governmental organizations,Mr. Henning Stöcks,Velzen,Germany,Constituency - Research and independent


In [25]:
D1.to_csv("V2_18topics_COMPLETE_DATA.csv")